In [ ]:
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import time
from collections import deque
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from ipywidgets import IntSlider, Dropdown, FloatSlider, ToggleButtons, Button, HBox, VBox, Output, Layout
from tqdm.notebook import tqdm
import base64
import json
from datetime import datetime
from tkinter import Tk, filedialog
import plotly.graph_objects as go

from utils.feature_extraction import extract_features, extract_live_features, circular_mean, circular_std
from utils.clustering import perform_clustering, update_live_model, find_optimal_clusters
from utils.visualization import (
    plot_cluster_metrics,
    show_cluster_visualization,
    show_feature_distributions,
    show_cluster_timeline,
    show_movement_directions,
    show_motion_heatmaps,
    update_live_visualization
)
from utils.webcam_utils import capture_frame, compute_optical_flow
from config import Config


class HumanMovementAnalyzer:
    def __init__(self):
        """Initialize the Human Movement Analyzer with default settings"""
        self._initialize_attributes()
        self._setup_dashboard()

    def _initialize_attributes(self):
        """Initialize all class attributes with default values"""
        # Analysis parameters
        self.features = None
        self.cluster_results = {}
        self.optimal_k = Config.DEFAULT_CLUSTERS
        self.current_mode = "video_upload"
        self.is_running = False
        self.analysis_active = False
        self.video_processing = False

        # Data storage
        self.magnitudes = []
        self.angles = []
        self.video_fps = Config.DEFAULT_FPS
        self.frame_timestamps = []

        # Buffers for live analysis
        self.frame_buffer = deque(maxlen=Config.FRAME_BUFFER_SIZE)
        self.feature_buffer = deque(maxlen=Config.FEATURE_BUFFER_SIZE)
        self.cluster_history = deque(maxlen=Config.CLUSTER_HISTORY_SIZE)
        self.prev_frame = None
        self.frame_count = 0
        self.start_time = time.time()

        # UI elements
        self.upload_button = None
        self.start_button = None
        self.stop_button = None
        self.progress_bar = None
        self.status_output = None
        self.stats_display = None
        self.results_display = None

    def _setup_dashboard(self):
        """Initialize the dashboard layout and controls"""
        self._display_header()
        self._setup_controls()
        self._setup_mode_specific_ui()

    def _display_header(self):
        """Display the dashboard header with styling"""
        header = widgets.HTML("""
        <div style="
            background: linear-gradient(135deg, #6e8efb, #a777e3);
            color: white;
            padding: 20px;
            border-radius: 5px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        ">
            <h1 style="margin: 0;">Professional Human Movement Analysis</h1>
            <p style="margin: 5px 0 0;">Advanced motion pattern detection and clustering</p>
        </div>
        """)
        display(header)

    def _setup_controls(self):
        """Setup the main control panel with interactive widgets"""
        # Mode selection
        self.mode_selector = ToggleButtons(
            options=[('Video Upload', 'video_upload'), ('Live Webcam', 'live_webcam')],
            description='Analysis Mode:',
            disabled=False,
            button_style='',
            style={'description_width': 'initial'}
        )

        # Analysis parameters
        self.cluster_slider = IntSlider(
            value=Config.DEFAULT_CLUSTERS,
            min=2,
            max=10,
            step=1,
            description='Max Clusters:',
            continuous_update=False,
            style={'description_width': 'initial'}
        )

        self.sensitivity_slider = FloatSlider(
            value=Config.DEFAULT_SENSITIVITY,
            min=0.1,
            max=1.0,
            step=0.1,
            description='Motion Sensitivity:',
            continuous_update=False,
            style={'description_width': 'initial'}
        )

        self.algorithm_selector = Dropdown(
            options=['K-Means', 'Bayesian GMM', 'Auto-Detect'],
            value=Config.DEFAULT_ALGORITHM,
            description='Clustering Algorithm:',
            disabled=False,
            style={'description_width': 'initial'}
        )

        # Create widgets for control labels
        analysis_label = widgets.HTML("<b>Analysis Parameters</b>")

        # Display controls
        display(self.mode_selector)
        display(analysis_label)

        controls_box = VBox([
            self.cluster_slider,
            self.sensitivity_slider,
            self.algorithm_selector
        ], layout=Layout(width='100%', margin='0 0 20px 0'))

        display(controls_box)

        # Bind events
        self.mode_selector.observe(self._on_mode_change, names='value')

    def _setup_mode_specific_ui(self):
        """Setup UI elements specific to each analysis mode"""
        if self.current_mode == "video_upload":
            self._setup_video_upload_ui()
        else:
            self._setup_live_analysis_ui()

    def _setup_video_upload_ui(self):
        """Setup the video upload interface"""
        video_label = widgets.HTML("<b>Video Analysis</b>")

        info_text = widgets.HTML("""
        <div style="
            border: 1px solid #ddd;
            padding: 15px;
            border-radius: 5px;
            margin-bottom: 15px;
            background-color: #f9f9f9;
        ">
            <h3 style="margin-top: 0;">Video Analysis</h3>
            <p>Upload a video file to analyze movement patterns</p>
            <div style="
                border: 1px solid #ffcc00;
                background-color: #fff8e1;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
            ">
                <strong>Note:</strong> For best results, use videos with clear movement patterns.
                Supported formats: MP4, AVI, MOV (H.264 codec recommended).
            </div>
        </div>
        """)

        # Create upload button
        self.upload_button = widgets.Button(
            description="Upload & Analyze Video",
            button_style='success',
            icon='upload',
            layout=Layout(width='200px', height='40px')
        )

        self.progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Progress:',
            bar_style='info',
            style={'bar_color': '#4285F4'},
            layout=Layout(width='100%')
        )

        self.status_output = Output(layout={'border': '1px solid #eee', 'max_height': '300px', 'overflow_y': 'auto'})
        self.results_display = Output(layout={'border': '1px solid #eee', 'max_height': '600px', 'overflow_y': 'auto'})

        # Button container
        button_box = HBox([self.upload_button], layout=Layout(justify_content='center', margin='10px 0'))

        # Progress container
        progress_label = widgets.HTML("<b>Processing Progress</b>")
        progress_container = VBox([
            progress_label,
            self.progress_bar,
            self.status_output
        ], layout=Layout(width='100%'))

        # Display all widgets
        display(VBox([
            video_label,
            info_text,
            button_box,
            progress_container,
            self.results_display
        ]))

        # Bind events
        self.upload_button.on_click(self._on_upload_button_clicked)

    def _setup_live_analysis_ui(self):
        """Setup the live analysis interface"""
        live_label = widgets.HTML("<b>Live Webcam Analysis</b>")

        info_text = widgets.HTML("""
        <div style="
            border: 1px solid #ddd;
            padding: 15px;
            border-radius: 5px;
            margin-bottom: 15px;
            background-color: #f9f9f9;
        ">
            <h3 style="margin-top: 0;">Live Movement Analysis</h3>
            <p>Analyze movement patterns in real-time using your webcam</p>
            <div style="
                border: 1px solid #ffcc00;
                background-color: #fff8e1;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
            ">
                <strong>Note:</strong> You'll need to grant camera permissions when prompted.
                For best results, ensure good lighting and clear movement in frame.
            </div>
        </div>
        """)

        # Create control buttons
        self.start_button = widgets.Button(
            description="Start Analysis",
            button_style='success',
            icon='play',
            layout=Layout(width='150px', height='40px')
        )

        self.stop_button = widgets.Button(
            description="Stop Analysis",
            button_style='danger',
            icon='stop',
            disabled=True,
            layout=Layout(width='150px', height='40px')
        )

        self.status_output = Output(layout={'border': '1px solid #eee', 'max_height': '300px', 'overflow_y': 'auto'})
        self.stats_display = Output(layout={'border': '1px solid #eee', 'max_height': '200px', 'overflow_y': 'auto'})

        # Button container
        button_box = HBox([self.start_button, self.stop_button],
                          layout=Layout(justify_content='center', margin='10px 0'))

        # Display all widgets
        display(VBox([
            live_label,
            info_text,
            button_box,
            self.status_output,
            self.stats_display
        ]))

        # Bind events
        self.start_button.on_click(self._on_start_button_clicked)
        self.stop_button.on_click(self._on_stop_button_clicked)

    def _on_mode_change(self, change):
        """Handle mode selection change"""
        self.current_mode = change['new']
        clear_output(wait=True)
        self._setup_dashboard()

    def _on_upload_button_clicked(self, b):
        """Handle video upload button click"""
        with self.status_output:
            clear_output()
            print("Preparing for video upload...")

        # Clear any previous analysis
        self._reset_analysis()

        # Start upload process
        self._upload_and_process_video()

    def _upload_and_process_video(self):
        """Handle video upload and processing using file dialog"""
        try:
            # Hide the root window
            root = Tk()
            root.withdraw()

            # Show file dialog
            video_path = filedialog.askopenfilename(
                title="Select video file",
                filetypes=[("Video files", "*.mp4 *.avi *.mov"), ("All files", "*.*")]
            )

            if not video_path:
                with self.status_output:
                    display(widgets.HTML("<div style='color:red'>No file was selected. Please try again.</div>"))
                return

            with self.status_output:
                clear_output()
                print(f"Selected file: {video_path}")
                display(widgets.HTML(f"<div style='color:green'>Selected file: {video_path}</div>"))
                print("Starting video processing...")

            # Process the video file
            self._process_video_file(video_path)

        except Exception as e:
            with self.status_output:
                display(widgets.HTML(f"<div style='color:red'>Error during file selection: {str(e)}</div>"))

    def _process_video_file(self, video_path):
        """Process uploaded video file with robust error handling"""
        self.video_processing = True
        cap = None

        try:
            with self.status_output:
                print(f"Opening video file: {video_path}")

            # Open video file
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                raise ValueError(f"Could not open video file: {video_path}")

            # Get video properties
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            if fps == 0:
                fps = Config.DEFAULT_FPS  # default assumption

            self.video_fps = fps
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            with self.status_output:
                print(f"Video properties: {frame_width}x{frame_height} @ {fps:.1f}fps, {total_frames} frames")
                print("Processing frames...")

            # Initialize containers
            frames = []
            flows = []
            self.magnitudes = []
            self.angles = []
            self.frame_timestamps = []

            # Setup progress bar
            self.progress_bar.max = total_frames
            self.progress_bar.value = 0
            self.progress_bar.description = 'Processing:'

            # Read first frame
            ret, prev_frame = cap.read()
            if not ret:
                raise ValueError("Could not read first frame")

            prev_gray = cv2.cvtColor(cv2.resize(prev_frame,
                                                (Config.RESIZE_WIDTH, Config.RESIZE_HEIGHT)), cv2.COLOR_BGR2GRAY)
            frames.append(prev_gray)
            self.progress_bar.value += 1

            # Process subsequent frames
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Process frame
                current_gray = cv2.cvtColor(cv2.resize(frame,
                                                       (Config.RESIZE_WIDTH, Config.RESIZE_HEIGHT)), cv2.COLOR_BGR2GRAY)
                frames.append(current_gray)

                # Compute optical flow
                flow = cv2.calcOpticalFlowFarneback(
                    prev_gray, current_gray, None,
                    0.5, 3, 15, 3, 5, 1.2, 0
                )

                magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

                flows.append((magnitude, angle))
                self.magnitudes.append(magnitude)
                self.angles.append(angle)
                self.frame_timestamps.append(self.progress_bar.value / fps)

                prev_gray = current_gray
                self.progress_bar.value += 1

                # Update status periodically
                if self.progress_bar.value % 50 == 0:
                    with self.status_output:
                        print(f"Processed {self.progress_bar.value}/{total_frames} frames")

            # Release video capture
            cap.release()

            with self.status_output:
                print("Finished processing frames. Extracting features...")

            # Extract features
            self.features = extract_features(flows)

            with self.status_output:
                print(f"Extracted {len(self.features)} feature vectors")
                print("Performing clustering analysis...")

            # Perform clustering
            self._analyze_movements()

            with self.status_output:
                print("Analysis complete. Generating visualizations...")

            # Show results
            self._show_video_results(video_path)

            with self.status_output:
                display(widgets.HTML("<div style='color:green'>Analysis completed successfully!</div>"))

        except Exception as e:
            with self.status_output:
                display(widgets.HTML(f"<div style='color:red'>Error processing video: {str(e)}</div>"))

            if cap and cap.isOpened():
                cap.release()

        finally:
            self.video_processing = False

    def _analyze_movements(self):
        """Perform movement clustering analysis"""
        try:
            # Perform clustering based on selected algorithm
            self.cluster_results, self.scaler = perform_clustering(
                self.features,
                algorithm=self.algorithm_selector.value,
                n_clusters=self.cluster_slider.value
            )

            # If auto-detect was used, update the optimal_k
            if self.algorithm_selector.value == 'Auto-Detect':
                self.optimal_k = len(np.unique(self.cluster_results['KMeans']))

            with self.status_output:
                if 'KMeans' in self.cluster_results:
                    print(f"K-Means clustering completed with {self.optimal_k} clusters")
                    if 'metrics' in self.cluster_results:
                        metrics = self.cluster_results['metrics']
                        print(f"Silhouette Score: {metrics['silhouette']:.3f}")
                        print(f"Calinski-Harabasz Index: {metrics['calinski']:.1f}")
                        print(f"Davies-Bouldin Index: {metrics['davies']:.3f}")

                if 'BayesianGMM' in self.cluster_results:
                    n_components = len(np.unique(self.cluster_results['BayesianGMM']))
                    print(f"Bayesian GMM completed with {n_components} components")

        except Exception as e:
            with self.status_output:
                display(widgets.HTML(f"<div style='color:red'>Error during clustering: {str(e)}</div>"))

    def _show_video_results(self, video_path):
        """Display comprehensive results for video analysis"""
        with self.results_display:
            clear_output(wait=True)

            # Create all widgets first
            results_header = widgets.HTML("<b>Analysis Results</b>")

            duration = len(self.features) / self.video_fps
            summary_html = widgets.HTML(f"""
            <div style="
                border: 1px solid #ddd;
                padding: 15px;
                border-radius: 5px;
                margin-bottom: 15px;
                background-color: #f9f9f9;
            ">
                <h3 style="margin-top: 0;">Analysis Summary</h3>
                <p><strong>Video:</strong> {video_path}</p>
                <p><strong>Duration:</strong> {duration:.1f} seconds</p>
                <p><strong>Frames analyzed:</strong> {len(self.features)}</p>
                <p><strong>Optimal clusters:</strong> {self.optimal_k}</p>
            </div>
            """)

            # Display the widgets
            display(VBox([results_header, summary_html]))

            # Show all visualizations
            if 'KMeans' in self.cluster_results:
                self._show_cluster_visualization()
                self._show_feature_distributions()
                self._show_cluster_timeline()
                self._show_cluster_statistics()
                self._show_movement_directions()
                self._show_motion_heatmaps()

    def _show_cluster_visualization(self):
        """Show cluster visualization in reduced dimension space"""
        with self.results_display:
            display(widgets.HTML("<b>Cluster Visualization</b>"))

            # Get scaled features
            X = self.scaler.transform(self.features)

            # Generate and show the visualization
            fig = show_cluster_visualization(X, self.cluster_results['KMeans'])
            display(fig)

    def _show_feature_distributions(self):
        """Show distributions of key features by cluster"""
        with self.results_display:
            display(widgets.HTML("<b>Feature Distributions</b>"))

            # Select key features to show
            features_to_show = [
                'mag_mean', 'mag_std', 'mag_max',
                'ang_mean', 'ang_std',
                'flow_magnitude', 'flow_direction'
            ]

            # Generate and show visualizations
            figs = show_feature_distributions(self.features, self.cluster_results['KMeans'], features_to_show)
            for fig in figs:
                display(fig)

    def _show_cluster_timeline(self):
        """Show cluster assignments over time"""
        with self.results_display:
            display(widgets.HTML("<b>Cluster Timeline</b>"))

            # Generate and show the timeline
            fig = show_cluster_timeline(
                self.frame_timestamps,
                self.cluster_results['KMeans'],
                self.features['mag_mean'],
                self.features['flow_direction']
            )
            display(fig)

    def _show_cluster_statistics(self):
        """Show statistics for each cluster"""
        with self.results_display:
            display(widgets.HTML("<b>Cluster Statistics</b>"))

            stats = []

            for cluster_id in np.unique(self.cluster_results['KMeans']):
                mask = self.cluster_results['KMeans'] == cluster_id
                cluster_data = self.features[mask]

                # Calculate statistics
                duration = len(cluster_data) / self.video_fps
                mag_mean = cluster_data['mag_mean'].mean()
                mag_std = cluster_data['mag_std'].mean()
                direction = np.rad2deg(circular_mean(cluster_data['flow_direction']))

                stats.append({
                    'Cluster': cluster_id,
                    'Frames': len(cluster_data),
                    'Duration (s)': f"{duration:.1f}",
                    'Avg Magnitude': f"{mag_mean:.2f}",
                    'Magnitude STD': f"{mag_std:.2f}",
                    'Dominant Direction': f"{direction:.1f}°"
                })

            stats_df = pd.DataFrame(stats)

            fig = go.Figure(data=[go.Table(
                header=dict(
                    values=list(stats_df.columns),
                    fill_color='#6e8efb',
                    align='center',
                    font=dict(color='white', size=12)
                ),
                cells=dict(
                    values=[stats_df[col] for col in stats_df.columns],
                    fill_color='lavender',
                    align='center'
                )
            )])

            fig.update_layout(
                title='Cluster Statistics Summary',
                margin=dict(l=10, r=10, b=10, t=40)
            )

            display(fig)

    def _show_movement_directions(self):
        """Show polar plot of movement directions"""
        with self.results_display:
            display(widgets.HTML("<b>Movement Directions</b>"))

            # Generate and show the visualization
            fig = show_movement_directions(
                self.cluster_results['KMeans'],
                self.features['flow_direction'],
                self.features['mag_mean']
            )
            display(fig)

    def _show_motion_heatmaps(self):
        """Show heatmaps of motion patterns"""
        with self.results_display:
            display(widgets.HTML("<b>Motion Heatmaps</b>"))

            # Generate and show heatmaps
            figs = show_motion_heatmaps(self.magnitudes, self.cluster_results['KMeans'])
            for fig in figs:
                display(fig)

    def _on_start_button_clicked(self, b):
        """Handle start button click for live analysis"""
        self.start_button.disabled = True
        self.stop_button.disabled = False

        with self.status_output:
            clear_output()
            print("Starting live analysis...")
            print("Please grant camera permissions when prompted")

        # Reset analysis state
        self._reset_analysis()
        self.is_running = True
        self.analysis_active = True

        # Start live analysis in a separate thread
        import threading
        analysis_thread = threading.Thread(target=self._run_live_analysis)
        analysis_thread.start()

    def _on_stop_button_clicked(self, b):
        """Handle stop button click for live analysis"""
        with self.status_output:
            print("Stopping analysis...")

        self.is_running = False
        self.start_button.disabled = False
        self.stop_button.disabled = True

    def _run_live_analysis(self):
        """Run live webcam analysis with robust error handling"""
        try:
            # Create visualization figure
            plt.figure(figsize=(14, 6), num='Live Movement Analysis')
            plt.ion()

            # Initialize frame counter
            self.frame_count = 0
            self.start_time = time.time()

            while self.is_running:
                # Capture frame from webcam
                frame = capture_frame()
                if frame is None:
                    time.sleep(0.1)
                    continue

                # Process frame
                current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                current_gray = cv2.resize(current_gray, (Config.RESIZE_WIDTH, Config.RESIZE_HEIGHT))

                if self.prev_frame is not None:
                    # Compute optical flow
                    magnitude, angle, _ = compute_optical_flow(self.prev_frame, frame)

                    if magnitude is not None:
                        # Apply sensitivity threshold
                        magnitude_threshold = np.percentile(magnitude, 100 * (1 - self.sensitivity_slider.value))
                        magnitude[magnitude < magnitude_threshold] = 0

                        # Extract features
                        features = extract_live_features(magnitude, angle)
                        self.feature_buffer.append(features)

                        # Update clustering model periodically
                        if self.frame_count % 10 == 0 and len(self.feature_buffer) > 15:
                            self.kmeans, self.scaler = update_live_model(
                                np.array(self.feature_buffer),
                                algorithm=self.algorithm_selector.value,
                                n_clusters=self.cluster_slider.value
                            )

                        # Predict current cluster
                        if len(self.feature_buffer) > 0 and hasattr(self.scaler, 'mean_'):
                            current_features = np.array([features])
                            scaled = self.scaler.transform(current_features)
                            current_cluster = self.kmeans.predict(scaled)[0]
                        else:
                            current_cluster = 0

                        # Update visualization
                        update_live_visualization(
                            frame, magnitude, angle, current_cluster,
                            self.cluster_history,
                            self.frame_count / (time.time() - self.start_time),
                            self.cluster_slider.value
                        )

                        # Update cluster history
                        self.cluster_history.append(current_cluster)

                self.prev_frame = current_gray.copy()
                self.frame_count += 1

                # Limit frame rate
                time.sleep(0.05)

        except Exception as e:
            with self.status_output:
                display(widgets.HTML(f"<div style='color:red'>Error during live analysis: {str(e)}</div>"))

        finally:
            plt.ioff()
            plt.close()
            self.is_running = False
            self.analysis_active = False

            with self.status_output:
                print("Live analysis stopped")
                print(f"Processed {self.frame_count} frames")

    def _reset_analysis(self):
        """Reset the analysis state"""
        self.features = None
        self.cluster_results = {}
        self.magnitudes = []
        self.angles = []
        self.frame_timestamps = []
        self.frame_count = 0
        self.start_time = time.time()

        # Reset buffers
        self.frame_buffer.clear()
        self.feature_buffer.clear()
        self.cluster_history.clear()
        self.prev_frame = None

        # Reset models
        self.scaler = StandardScaler()
        self.kmeans = MiniBatchKMeans(
            n_clusters=Config.DEFAULT_CLUSTERS,
            batch_size=100,
            random_state=42
        )

In [4]:
from analyzer import HumanMovementAnalyzer
analyzer = HumanMovementAnalyzer()  # This will now work properly

ToggleButtons(description='Analysis Mode:', options=(('Video Upload', 'video_upload'), ('Live Webcam', 'live_w…

TraitError: The 'children' trait of a VBox instance contains an Instance of a TypedTuple which expected a Widget, not the HTML <IPython.core.display.HTML object>.